In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

print(torch.__version__)
print(torch.cuda.is_available())

1.2.0
True


#### Check if is on CPU or GPU

```python
# Tensor
print(X.device)

# Model
print(next(model.parameters()).is_cuda)
```

# Train data by GPU

#### Sample Model

In [2]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)

        gpu_id = torch.cuda.current_device()
        gpu_name = torch.cuda.get_device_name()
        print(f"=== Using GPU{gpu_id} {gpu_name} ===")
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

In [3]:
input_size = 5
output_size = 2
model = Model(input_size, output_size)

#### Sample Dataset

In [4]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

In [5]:
data_size = 100
input_size = 5
batch_size = 30

random_dataset = RandomDataset(input_size, data_size)
rand_loader = DataLoader(
    dataset=random_dataset,
    batch_size=batch_size, shuffle=True)

In [6]:
random_dataset.data.shape

torch.Size([100, 5])

In [7]:
print(len(rand_loader))
for batch in rand_loader:
    print(batch.shape)

4
torch.Size([30, 5])
torch.Size([30, 5])
torch.Size([30, 5])
torch.Size([10, 5])


# Load to 1 GPU

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Use {device}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

Use cuda:0
Number of GPUs: 4


In [9]:
# Load model
model.to(device)

# Load data
for batch in rand_loader:
    # Load data
    Xb = batch.to(device)

    # Train
    yb_ = model(Xb)
    print("Outside: input size", Xb.size(),
            "output_size", yb_.size())

=== Using GPU0 Tesla K40m ===
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
=== Using GPU0 Tesla K40m ===
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
=== Using GPU0 Tesla K40m ===
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
=== Using GPU0 Tesla K40m ===
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


#### Save model - 1 GPU

In [10]:
# Check params
model.state_dict()

OrderedDict([('fc.weight',
              tensor([[ 0.0215,  0.1493, -0.3619,  0.4006, -0.3169],
                      [ 0.2177,  0.0907, -0.0291,  0.1594, -0.1887]], device='cuda:0')),
             ('fc.bias', tensor([0.3982, 0.4082], device='cuda:0'))])

In [11]:
# Save model
torch.save(model.state_dict(), 'mymodel.pt')

In [12]:
# Load model
model_load = Model(input_size, output_size)
model_load.load_state_dict(torch.load('mymodel.pt'))

<All keys matched successfully>

# Load to multi GPUs

In [13]:
# cuda:<num> - GPU<num> must be used
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Use {device}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

Use cuda:1
Number of GPUs: 4


In [14]:
# Load model
if torch.cuda.device_count() > 1:
    # Specify number of GPUs + GPU id by
    #   model = nn.DataParallel(model, device_ids=[0,1,2,3]).cuda()
    model = nn.DataParallel(model, device_ids=[1,2]).cuda()
model.to(device)

DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [15]:
# Load 1 batch of data -> different GPU
for batch in rand_loader:
    # Load data
    Xb = batch.to(device)

    # Train
    yb_ = model(Xb)
    print("Outside: input size", Xb.size(),
            "output_size", yb_.size())

=== Using GPU1 Tesla K40m ===
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
=== Using GPU2 Tesla K20m ===
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
=== Using GPU1 Tesla K40m ===
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
=== Using GPU2 Tesla K20m ===
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
=== Using GPU1 Tesla K40m ===
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
=== Using GPU2 Tesla K20m ===
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
=== Using GPU1 Tesla K40m ===
	In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
=== Using GPU2 Tesla K20m ===
	In Model: input size torch.

#### Save model - multi GPU

In [16]:
# Check params
model.state_dict()

OrderedDict([('module.fc.weight',
              tensor([[ 0.0215,  0.1493, -0.3619,  0.4006, -0.3169],
                      [ 0.2177,  0.0907, -0.0291,  0.1594, -0.1887]], device='cuda:1')),
             ('module.fc.bias', tensor([0.3982, 0.4082], device='cuda:1'))])

In [17]:
# Save model
torch.save(model.state_dict(), 'mymodel.pt')

In [18]:
# Load model
model_load = Model(input_size, output_size)

model_dic = torch.load('mymodel.pt')

from collections import OrderedDict
new_state_dict = OrderedDict() 
for k, v in model_dic.items(): 
    name = k.replace('module.', '')
    new_state_dict[name] = v

model_load.load_state_dict(new_state_dict)

<All keys matched successfully>